In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import cmudict
nltk.download('punkt')
nltk.download('cmudict')
import textstat

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chandrahas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\Chandrahas\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [29]:
df = pd.read_excel('Input.xlsx')
df.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


In [30]:
df1=pd.DataFrame()

In [31]:
stop_words = []
stop_words_files=['StopWords\StopWords_Auditor.txt','StopWords\StopWords_Currencies.txt','StopWords\StopWords_DatesandNumbers.txt','StopWords\StopWords_Generic.txt','StopWords\StopWords_GenericLong.txt','StopWords\StopWords_Geographic.txt','StopWords\StopWords_Names.txt']
for file in stop_words_files:
    with open(file, 'r') as f:
        stop_words += f.read().splitlines()


In [32]:
positive_words = {}
negative_words = {}
with open('MasterDictionary\positive-words.txt', 'r') as f:
    for line in f:
        if line not in stop_words:
            positive_words[line.strip()] = 1
with open(r'MasterDictionary\negative-words.txt', 'r') as f:
    for line in f:
        if line not in stop_words:
            negative_words[line.strip()] = 1

In [33]:
df['POSITIVE SCORE'] = None
df['NEGATIVE SCORE'] = None
df['POLARITY SCORE'] = None
df['SUBJECTIVITY SCORE'] = None
df['AVG SENTENCE LENGTH'] = None
df['PERCENTAGE OF COMPLEX WORDS'] = None
df['FOG INDEX'] = None
df['AVG NUMBER OF WORDS PER SENTENCE']=None
df['COMPLEX WORD COUNT']=None
df['WORD COUNT']=None
df['SYLLABLE PER WORD']=None
df['PERSONAL PRONOUNS']=None
df['AVG WORD LENGTH']=None

In [34]:
with open('URL_ID.txt', 'w', encoding='utf-8') as f:
    for index, row in df.iterrows():
        # Get the URL from the current row
        url = row['URL']
        
        # Make a request to the URL and get the HTML content
        response = requests.get(url, timeout=100)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract the article title and content from the HTML content
        title = soup.find('title').text
        content = ''
        for paragraph in soup.find_all('p'):
            content += paragraph.text + '\n'
        
        #write the content to a new dataframe
        df1.at[index, 'URL'] = url
        df1.at[index, 'title'] = title
        df1.at[index, 'content'] = content

        # Write the extracted content to the text file
        f.write(content)
        f.write('\n\n')  # Add a separator between articles

        # Clean the text by removing stop words
        tokens = word_tokenize(content.lower())
        filtered_tokens = [token for token in tokens if token not in stop_words]
        cleaned_content = ' '.join(filtered_tokens)
        total_words = len(filtered_tokens)

        # Perform sentiment analysis on the article content
        positive_score = sum([positive_words.get(word, 0) for word in cleaned_content.split()])
        negative_score = sum([negative_words.get(word, 0) for word in cleaned_content.split()])
        polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
        subjectivity_score = (positive_score + negative_score) / ((total_words) + 0.000001)
        personal_pronouns_list = ['I', 'me', 'my', 'mine', 'myself','you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself',
                            'it', 'its', 'itself', 'we', 'us', 'our', 'ours', 'ourselves', 'they', 'them', 'their', 'theirs', 'themselves']

        # Analysis of Readability
        sentences = sent_tokenize(content)
        total_sentences = len(sentences)
        total_words = len(filtered_tokens)
        complex_words = 0
        total_syllables = 0
        personal_pronouns = 0
        for word in filtered_tokens:
            # Count syllables in each word
            syllables = textstat.syllable_count(word)
            total_syllables += syllables
            
            # Check if the word is a complex word
            if syllables > 2:
                complex_words += 1
                
            # Check if the word is a personal pronoun
            if word in personal_pronouns_list:
                personal_pronouns += 1

        # Calculate the average sentence length
        avg_sentence_length = total_words / total_sentences

        # Calculate the percentage of complex words
        percent_complex_words = (complex_words / total_words) * 100

        # Calculate the Fog Index
        fog_index = 0.4 * (avg_sentence_length + percent_complex_words)

        # Calculate the average number of words per sentence
        avg_words_per_sentence = total_words / total_sentences

        # Calculate the average word length
        total_characters = sum(len(word) for word in filtered_tokens)
        avg_word_length = total_characters / total_words

        # Update the dataframe with the sentiment analysis results
        df.at[index, 'POSITIVE SCORE'] = positive_score
        df.at[index, 'NEGATIVE SCORE'] = negative_score
        df.at[index, 'POLARITY SCORE'] = polarity_score
        df.at[index, 'SUBJECTIVITY SCORE'] = subjectivity_score
        df.at[index, 'AVG SENTENCE LENGTH'] = avg_sentence_length
        df.at[index, 'PERCENTAGE OF COMPLEX WORDS'] = percent_complex_words
        df.at[index, 'FOG INDEX'] = fog_index
        df.at[index, 'AVG NUMBER OF WORDS PER SENTENCE'] = avg_words_per_sentence
        df.at[index, 'COMPLEX WORD COUNT'] = complex_words
        df.at[index, 'WORD COUNT'] = total_words
        df.at[index, 'SYLLABLE PER WORD'] = total_syllables / total_words
        df.at[index, 'PERSONAL PRONOUNS'] = personal_pronouns
        df.at[index, 'AVG WORD LENGTH'] = avg_word_length


In [38]:
df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,70,37,0.308411,0.073238,17.817073,27.036277,17.94134,17.817073,395,1461,1.854894,0,6.326489
1,38,https://insights.blackcoffer.com/what-if-the-c...,64,41,0.219048,0.099432,12.27907,20.359848,13.055567,12.27907,215,1056,1.574811,0,5.493371
2,39,https://insights.blackcoffer.com/what-jobs-wil...,69,41,0.254545,0.084421,14.318681,27.24482,16.6254,14.318681,355,1303,1.848043,0,6.227936
3,40,https://insights.blackcoffer.com/will-machine-...,67,30,0.381443,0.088262,11.214286,22.11101,13.330118,11.214286,243,1099,1.66242,0,5.683348
4,41,https://insights.blackcoffer.com/will-ai-repla...,62,26,0.409091,0.070064,15.13253,23.089172,15.288681,15.13253,290,1256,1.709395,0,5.863854


In [36]:
df1.head()

,URL,title,content
0,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes |...,Ranking customer behaviours for business strat...
1,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creato...,Ranking customer behaviours for business strat...
2,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Ranking customer behaviours for business strat...
3,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,Ranking customer behaviours for business strat...
4,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us? | Blackcof...,Ranking customer behaviours for business strat...


In [37]:
#write this df to xslx format
df.to_excel('Output Data Structure.xlsx', index=False)